In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Crohns_Disease/GSE186582'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from intestinal mucosa of patients with Crohn disease"
!Series_summary	"We used microarrays to detail the global signature of gene expression underlying endoscopic recurrence of CD and identified distinct gene signature predicting postoperative recurrence."
!Series_overall_design	"Ileal samples from Crohn's disease patients and healthy samples from non-inflammatory controls were collected for RNA extraction and hybridization on Affymetrix microarrays. Inclusion criteria were age >18 years, ileal or ileocolonic CD and indication of CD‐related intestinal surgery. Endoscopic recurrence was defined by the presence of any ulcerated lesions at the anastomosis and/or on the neo-terminal ileum (Rutgeerts score > i0). Five hundred and twenty samples (520) were collected from the inflamed ileum (M0I) and the ileal margin (M0M) at time of surgery, and during post-operative endoscopy six month later (M6). We also collected 25 ileal non-IBD con

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True  # Based on the Series title and summary, it contains gene expression data
trait_row = 4  # The 'rutgeerts' key could represent Crohn's Disease status
age_row = None  # No age-related key found; Age >18 was a criterion but not explicitly listed
gender_row = 1  # Available under key 1

# Define functions to convert the data
def convert_trait(value):
    if 'i0' in value or '0' in value:
        return 0
    return 1

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except:
        return None

def convert_gender(value):
    gender = value.split(': ')[1]
    if gender == "Male":
        return 1
    if gender == "Female":
        return 0
    return None

# Save cohort information
save_cohort_info('GSE186582', './preprocessed/Crohns_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Crohns_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Crohns_Disease/trait_data/GSE186582.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM5656170': [0, 0], 'GSM5656171': [0, 0], 'GSM5656172': [1, 0], 'GSM5656173': [1, 0], 'GSM5656174': [1, 0], 'GSM5656175': [1, 1], 'GSM5656176': [1, 1], 'GSM5656177': [1, 1], 'GSM5656178': [0, 1], 'GSM5656179': [0, 1], 'GSM5656180': [1, 1], 'GSM5656181': [1, 0], 'GSM5656182': [1, 0], 'GSM5656183': [1, 0], 'GSM5656184': [1, 1], 'GSM5656185': [1, 1], 'GSM5656186': [1, 1], 'GSM5656187': [1, 1], 'GSM5656188': [1, 1], 'GSM5656189': [1, 0], 'GSM5656190': [1, 1], 'GSM5656191': [1, 1], 'GSM5656192': [1, 1], 'GSM5656193': [0, 0], 'GSM5656194': [0, 0], 'GSM5656195': [0, 0], 'GSM5656196': [0, 0], 'GSM5656197': [1, 0], 'GSM5656198': [1, 0], 'GSM5656199': [1, 0], 'GSM5656200': [1, 1], 'GSM5656201': [1, 1], 'GSM5656202': [1, 1], 'GSM5656203': [0, 1], 'GSM5656204': [0, 1], 'GSM5656205': [0, 1], 'GSM5656206': [1, 1], 'GSM5656207': [1, 1], 'GSM5656208': [1, 1], 'GSM5656209': [1, 1], 'GSM5656210': [1, 1], 'GSM5656211': [0, 0], 'GSM5656212': [0, 0], 'GSM5656213': [1, 0], 'GSM5656214': [1, 0], 'GSM56562

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1053_at', '121_at', '1316_at', '1405_i_at', '1487_at', '1552256_a_at',
       '1552257_a_at', '1552258_at', '1552266_at', '1552269_at',
       '1552272_a_at', '1552274_at', '1552277_a_at', '1552280_at',
       '1552281_at', '1552286_at', '1552287_s_at', '1552289_a_at',
       '1552293_at', '1552296_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:

# 1. Identify the dictionary keys for probe IDs and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the mapping between probe IDs and genes using the 'get_gene_mapping' function
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Crohns_Disease/gene_data/GSE186582.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Crohns_Disease')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE186582', './preprocessed/Crohns_Disease/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Crohns_Disease/GSE186582.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Crohns_Disease', the least common label is '0.0' with 138 occurrences. This represents 28.22% of the dataset.
The distribution of the feature 'Crohns_Disease' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 243 occurrences. This represents 49.69% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

